In [1]:
import numpy as np
from enum import Enum
from copy import copy, deepcopy
from collections import deque

import matplotlib.pyplot as plt 
%matplotlib notebook

from scipy.ndimage.filters import uniform_filter1d


In [2]:
CALORIES_STEP_SIZE=250

class Goal(Enum):
    LOSS = -CALORIES_STEP_SIZE
    MAINTAIN = 0
    GAIN = CALORIES_STEP_SIZE

In [3]:
class UnhandledCaseException(Exception):
    pass

In [11]:
def is_close(a, b, close=150):
    if abs(a - b) < close:
        return True
    return False

In [13]:
class State:
    pass

class Sequence:
    def __init__(self):
        self.state = {
            "goal": objective,
            "age_in_weeks": 0,
            "phase": Phase.LOSS,
            "dej": dej,
            "mb": mb,
            "aca": aca,
            "weight": 80,
            "weeks_in_phase": 0,
        }
        self.history = deque(maxlen=10)
        
    def get_weight_last_n_weeks(self, n=3):
        if len(self.history) >= n:
            return [x["weight"] for x in self.history][-n:]

In [17]:
class Phase(Enum):
    LOSS = 0
    STOP = 1
    MAINTAINANCE = 2 
    GAIN = 3

In [18]:
s = Sequence()
s.state

{'goal': <Goal.LOSS: -250>,
 'age_in_weeks': 0,
 'phase': <Phase.LOSS: 0>,
 'dej': 1900,
 'mb': 1400,
 'aca': 2000,
 'weight': 80,
 'weeks_in_phase': 0}

In [19]:
s.get_weight_last_n_weeks()

In [25]:
def compute_dej(weight, height, age):
    return 1.8 * 191 * (1.083 * weight**0.48) * (height**0.50) * (age**-0.13)

# Simulation 

In [26]:
def next_weight(aca, dej, sigma=0.7):
    return np.random.normal(0, sigma) + ((aca - dej) * 7) / 7000 

In [27]:
initial_weight = 80

n_weeks = 100

delta_w = np.random.normal(size=100)

weights = np.cumsum(delta_w)
height = 1.80
age = 30

dej_hat = compute_dej(initial_weight, height, age)
aca = dej_hat



## Build muscle

In [28]:
def gain(x):
    if x > 0.5: 
        return 0
    if x <= 0.5:
        return CALORIES_STEP_SIZE

def maintain(x):
    if x > -0.5 and x < 0.5: 
        return 0
    if x > 0.5:
        return -CALORIES_STEP_SIZE
    if x < 0.5:
        return CALORIES_STEP_SIZE

In [29]:
weight_history = []
aca_history = []

weight_history.append(initial_weight)
aca_history.append(aca)
phases_history = [Phase.GAIN]

for i in range(n_weeks):
    delta_w = next_weight(aca, dej_hat)
    delta_cal = 0
    new_weight = weight_history[-1] + delta_w
    weight_history.append(new_weight)
    
    
    current_phase = phases_history[-1]
    duration = 1
    for i in range(0, len(phases_history)):
        if phases_history[-i-1] == current_phase:
            duration += 1
        else:
            break
            
    #print(duration)
    
    if current_phase == Phase.GAIN and duration <= 16:
        new_phase = Phase.GAIN
        
    
    if current_phase == Phase.GAIN and duration > 16:
        new_phase = Phase.STOP
        duration = 1
        
    if current_phase == Phase.STOP:
        if is_close(dej_hat, aca, close=200):
            new_phase = Phase.MAINTAINANCE
            phases_history.append(new_phase)
         
    if new_phase == Phase.MAINTAINANCE:
        if len(weight_history) > 3:
            weight_derivative = np.gradient(weight_history)
            y = uniform_filter1d(weight_derivative, size=3)
            delta_cal = maintain(y[-1])
    
    if new_phase == Phase.GAIN:
        if len(weight_history) > 3:
            weight_derivative = np.gradient(weight_history)
            y = uniform_filter1d(weight_derivative, size=3)
            delta_cal = gain(y[-1])
    
    if new_phase == Phase.STOP:
        if duration == 1:
            dej_hat = compute_dej(weight_history[-1], height, age)
        #print(dej_hat)
        if not is_close(dej_hat, aca, close=200):
            delta_cal = -CALORIES_STEP_SIZE
    
    phases_history.append(new_phase)
    aca = aca + delta_cal
    aca_history.append(aca)
    
#    print(aca)

In [33]:
fig = plt.figure(figsize=(8,8))

ax1 = fig.add_subplot(311)
ax1.set_ylabel('weight')
ax1.set_xlabel('# of weeks')
ax1.set_title('weight evolution across time')
ax1.plot(weight_history)


ax2 = fig.add_subplot(312)
ax2.set_ylabel('ACA')
ax2.set_xlabel('# of weeks')
ax2.set_title('ACA evolution across time')
ax2.plot(aca_history)

ax3 = fig.add_subplot(313)
ax3.set_ylabel('ACA')
ax3.set_xlabel('# of weeks')
ax3.set_title('phase evolution across time')
ax3.scatter(range(len(phases_history)), [x.value for x in phases_history]) 

plt.show()

<IPython.core.display.Javascript object>

## Lose Fat

In [31]:
def loss(x):
    if x < -0.5: 
        return 0
    if x > -0.5:
        return -CALORIES_STEP_SIZE

In [32]:
weight_history = []
aca_history = []

weight_history.append(initial_weight)
aca_history.append(aca)
phases_history = [Phase.LOSS]

for i in range(n_weeks):
    delta_w = next_weight(aca, dej_hat)
    delta_cal = 0
    new_weight = weight_history[-1] + delta_w
    weight_history.append(new_weight)
    
    
    current_phase = phases_history[-1]
    duration = 1
    for i in range(0, len(phases_history)):
        if phases_history[-i-1] == current_phase:
            duration += 1
        else:
            break
            
    #print(duration)
    
    if current_phase == Phase.LOSS and duration <= 16:
        new_phase = Phase.LOSS
        
    
    if current_phase == Phase.LOSS and duration > 16:
        new_phase = Phase.STOP
        duration = 1
        
    if current_phase == Phase.STOP:
        if is_close(dej_hat, aca, close=200):
            new_phase = Phase.MAINTAINANCE
            phases_history.append(new_phase)
         
    if new_phase == Phase.MAINTAINANCE:
        if len(weight_history) > 3:
            weight_derivative = np.gradient(weight_history)
            y = uniform_filter1d(weight_derivative, size=3)
            delta_cal = maintain(y[-1])
    
    if new_phase == Phase.LOSS:
        if len(weight_history) > 3:
            weight_derivative = np.gradient(weight_history)
            y = uniform_filter1d(weight_derivative, size=3)
            delta_cal = loss(y[-1])
    
    if new_phase == Phase.STOP:
        if duration == 1:
            dej_hat = compute_dej(weight_history[-1], height, age)
        #print(dej_hat)
        if not is_close(dej_hat, aca, close=200):
            delta_cal = +CALORIES_STEP_SIZE
    
    phases_history.append(new_phase)
    aca = aca + delta_cal
    aca_history.append(aca)
    
#    print(aca)

In [351]:
fig = plt.figure(figsize=(8,8))

ax1 = fig.add_subplot(311)
ax1.set_ylabel('weight')
ax1.set_xlabel('# of weeks')
ax1.set_title('weight evolution across time')
ax1.plot(weight_history)


ax2 = fig.add_subplot(312)
ax2.set_ylabel('ACA')
ax2.set_xlabel('# of weeks')
ax2.set_title('ACA evolution across time')
ax2.plot(aca_history)

ax3 = fig.add_subplot(313)
ax3.set_ylabel('ACA')
ax3.set_xlabel('# of weeks')
ax3.set_title('phase evolution across time')
ax3.scatter(range(len(phases_history)), [x.value for x in phases_history]) 

plt.show()

<IPython.core.display.Javascript object>